<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/Limpieza_AT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Faltan los datos de Guadalajara de AT**


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.0 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt

In [ ]:
# # Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
# folder_path = '/content/drive/My Drive/PFG_FASPAS/SAAF'
# docs_xlsx = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
# print(docs_xlsx)

# # Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
# folder_path_csv = '/content/drive/My Drive/PFG_FASPAS/Zonas_Prioritarias'
# docs_csv = [f for f in os.listdir(folder_path_csv) if f.endswith('.csv')]
# print(docs_csv)

In [ ]:
# dic_dataframes = {}
# dic_zonas = {}

# for doc in docs_xlsx:
#     entire_path = os.path.join(folder_path, doc)
#     df = pd.read_excel(entire_path)
#     # Uso el nombre del archivo como clave
#     dic_dataframes[doc] = df

# for doc in docs_csv:
#   entire_path = os.path.join(folder_path_csv, doc)
#   df = pd.read_csv(entire_path)
#   # Uso el nombre del archivo como clave
#   dic_zonas[doc] = df

# # Imprimir todas las claves
# for clave in dic_dataframes.keys():
#     print(clave)

In [ ]:
# CU_SAAF = dic_dataframes.get("CU_SAAF.xlsx")
# CU_AT = dic_dataframes.get("CU_AT.xlsx")
# CU_EMPLEO = dic_dataframes.get("CU_EMPLEO.xlsx")
# CU_LOG = dic_dataframes.get("CU_LOG.xlsx")
# # CU_SOCIOS = dic_dataframes.get("CU_SOCIOS.xlsx")

In [4]:
# CÓDIGO SOLO PARA EL CURRO (PQ NO PUEDO ACCEDER AL DRIVE)
# Primero cogemos todos los trimestres de CU para probar
CU_Trim1_2022_AT = pd.read_excel('/content/CU_Trim1_2022_AT.xlsx')
CU_Trim2_2022_AT = pd.read_excel('/content/CU_Trim2_2022_AT.xlsx')
CU_Trim3_2022_AT = pd.read_excel('/content/CU_Trim3_2022_AT.xlsx')
CU_Trim4_2022_AT = pd.read_excel('/content/CU_Trim4_2022_AT.xlsx')
CU_Trim1_2023_AT = pd.read_excel('/content/CU_Trim1_2023_AT.xlsx')
CU_Trim2_2023_AT = pd.read_excel('/content/CU_Trim2_2023_AT.xlsx')
CU_Trim3_2023_AT = pd.read_excel('/content/CU_Trim3_2023_AT.xlsx')
CU_Trim4_2023_AT = pd.read_excel('/content/CU_Trim4_2023_AT.xlsx')

In [6]:
CU_Trim1_2022_AT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 37 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Unnamed: 0                            7 non-null      object 
 1   Unnamed: 1                            7 non-null      object 
 2   Unnamed: 2                            7 non-null      object 
 3   Unnamed: 3                            7 non-null      object 
 4   Unnamed: 4                            7 non-null      object 
 5   Unnamed: 5                            35 non-null     object 
 6   Unnamed: 6                            35 non-null     object 
 7   Unnamed: 7                            7 non-null      object 
 8   Unnamed: 8                            7 non-null      object 
 9   Unnamed: 9                            7 non-null      object 
 10  Unnamed: 10                           7 non-null      object 
 11  Unnamed: 11          

In [7]:
# Definir la función que realiza las transformaciones
def transform_df(df):
    col_excluded = ['FECHA NACIMIENTO', 'FECHA DE NACIMIENTO', 'FECHA NAC']

    # Convertir todas las columnas de tipo object a mayúsculas, excepto las especificadas
    for column in df.columns:
        if df[column].dtype == 'object' and column not in col_excluded:
            df[column] = df[column].apply(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)

    # Ahora, aplicamos unidecode a los nombres de las columnas
    df.columns = [unidecode(col.upper()) for col in df.columns]

    return df

In [ ]:
CU_Trim1_2022_AT = transform_df(CU_Trim1_2022_AT)